# Usage of [`turboSETI`](https://github.com/UCBerkeleySETI/turbo_seti)
Updated 2020-08-06 by Shane Smith (ssmith3@hillsdale.edu)

---
## Intro
`turboSETI` is a Python tool used and developed by SETI researchers at the [Berkeley SETI Research Center](https://seti.berkeley.edu). The alogorithm searches for narrow band signals which have a doppler drift, a feature expected to be seen from an alien source with a non-zero acceleration relative to our recievers on Earth. <br/>

A basic introduction on installation and usage is given on `turboSETI`'s [README.md on GitHub](https://github.com/UCBerkeleySETI/turbo_seti/blob/master/README.md). This tutorial intends to give a more comprehensive look at the usage of the tools contained in `turboSETI`. <br/>

There are two ways of getting access to `turboSETI`.
1) If you are ssh'd into the BL data center, you can access a recent version by running `source /opt/conda/init.sh` in the terminal.
- A recent version of `turboSETI` should also be installed on the default Python3 kernel on the BL Jupyter Lab server.

2) You can create a new environment with the latest version of `turboSETI`

Here are some instructions on creating a new environment and adding it to a Jupyter Notebook:

First we'll need to set up an anaconda enviornment by running these commands in the bash terminal:
- `$ conda deactivate`
- `$ conda create -n turboSETI` 
    - Naming after -n is arbitrary, the name of your environment can be any name you want
- `$ conda activate turboSETI`
- `$ conda install pip`

Now let's install the required packages:
- `$ ~/.conda/envs/turboSETI/bin/pip install git+https://github.com/UCBerkeleySETI/blimpy` 
    - `~/.conda/envs/turboSETI/bin/pip` is the location of your pip. This may be different depending on your configuration.
    - `blimpy` is the file I/O for BL SETI. It will automatically gather all required packages for `turboSETI`
- `$ ~/.conda/envs/turboSETI/bin/pip install git+https://github.com/UCBerkeleySETI/turbo_seti`

Now we need to install our environment as an `IPython` kernel, so we can use it in Jupyter:
- `$ conda install -c anaconda ipykernel`
- `$ python -m ipykernel install --user --name=turboSETI`

A fresh version of `turboSETI` is now installed in the kernel `turboSETI`. You will have to restart Jupyter to see this kernel and be able to switch over (a quick refresh of the webpage should work).

## Creating a Search Routine
Now that we have `turboSETI` installed we can start a Doppler drift search on some data. First we want to create a search object using `turbo_seti.find_doppler.find_doppler.FindDoppler()`

In [1]:
from turbo_seti.find_doppler.find_doppler import FindDoppler
FindDoppler?

Init signature:
FindDoppler(
    datafile,
    max_drift,
    min_drift=0,
    snr=25.0,
    out_dir='./',
    coarse_chans=None,
    obs_info=None,
    flagging=None,
    n_coarse_chan=None,
)
Init docstring:
Initializes FindDoppler object

Args:
    datafile (string):  Inputted filename (.h5 or .fil)
    max_drift (float):  Max drift rate in Hz/second
    min_drift (int):    Min drift rate in Hz/second
    snr (float):        Signal to Noise Ratio - A ratio bigger than 1 to 1 has more signal than  noise
    out_dir (string):   Directory where output files should be placed. By default this is the
                        current working directory.
    coarse_chans (list(int)):  the inputted comma separated list of coarse channels to analyze, if any.
    obs_info (dict):     Used to hold info found on file, including info about pulsars, RFI, and SEFD
    flagging (bool):     Flags the edges of the PFF for BL data (with 3Hz res per channel)
    n_coarse_chan (int): Number of coarse chann

From the docstring, we see that we will need to find a .h5 or .fil file to perform a search on. 

We'll search some files that have been condensed to a single coarse channel and are routinely used for testing the code. They are located here:
http://blpd0.ssl.berkeley.edu/parkes_testing/ (total download size of 294 MB).

Typical SETI searches, such as [Price et. al]('https://arxiv.org/pdf/1906.07750.pdf'), have used drift rates of up to $\pm 4$Hz and a minimum S/N of 10. We'll adapt those parameters for this tutorial.


In [2]:
doppler = FindDoppler('diced_Parkes_57941_12846_HIP33499_S_fine.h5',
                      max_drift = 4,
                      snr = 10,       
                      out_dir = './' # This is where the turboSETI output files will be stored. "./" is the current directory
                     )

Now we run the search routine on the object we created:

In [3]:
doppler.search()

You will find that the search process can take many hours, especially when looking at multiple files that are Gigabytes in size. If you are doing your work on the BL servers or any other server system where you will be kicked out after an amount of time, I reccomend using either [tmux](https://github.com/tmux/tmux/wiki) or [screen](https://linuxize.com/post/how-to-use-linux-screen/). 

Terminal usage for `turboSETI` is as follows:

```
ssmith@blpc0:~$ source /opt/conda/init.sh
(base) ssmith@blpc0:~$ turboSETI -h
usage: turboSETI [-h] [-M MAX_DRIFT] [-s SNR] [-o OUT_DIR] [-l LOGLEVEL]
                 [-c COARSE_CHANS] [-n N_COARSE_CHAN] [-p N_PARALLEL]
                 filename

turboSETI doppler drift narrowband search utility.

positional arguments:
  filename              Name of filename to open (h5 or fil)

optional arguments:
  -h, --help            show this help message and exit
  -M MAX_DRIFT, --max_drift MAX_DRIFT
                        Set the drift rate to search. Unit: Hz/sec. Default:
                        10.0
  -s SNR, --snr SNR     SNR threshold. Default: 25.0
  -o OUT_DIR, --out_dir OUT_DIR
                        Location for output files. Default: local dir.
  -l LOGLEVEL, --loglevel LOGLEVEL
                        Specify log level (info, debug)
  -c COARSE_CHANS, --coarse_chans COARSE_CHANS
                        Comma separated list of coarse channels to analyze.
  -n N_COARSE_CHAN, --n_coarse_chan N_COARSE_CHAN
                        Number of coarse channels in file.
  -p N_PARALLEL, --n_parallel N_PARALLEL
                        Number of threads to run in parallel. Default to 1
                        (single threaded)
```

The parameters are similar to the initialization that we performed. One big difference between the terminal and Python is that the search process will start on it's own after you input your desired parameters. Another useful feature of the terminal is the ability to multi-thread with the `-p N_PARALLEL` option. This functionality does not currently exist in the Python interface.

`turboSETI` creates the following output files:
- `.h5` --- If your input file is a guppi raw or filterbank file, it will create a copy of it in hdf5 format
- `.dat` --- This is the most interesting file, it contains all of `turboSETI`'s hits
- `.log` --- This is the logged output of the search algorithm.

## Making a Pandas Dataframe
We can convert the `.dat` file into a [Pandas](https://pandas.pydata.org/) dataframe so it is easier to read:

In [4]:
from turbo_seti.find_event.find_event import make_table

df = make_table('diced_Parkes_57941_12846_HIP33499_S_fine.dat')
df

,TopHitNum,DriftRate,SNR,Freq,ChanIndx,FreqStart,FreqEnd,CoarseChanNum,FullNumHitsInRange,FileID,Source,MJD,RA,DEC,DELTAT,DELTAF,Hit_ID,status,in_n_ons,RFI_in_range
0,1,0.0,10.297262,2673.741800,34814,2673.740166,2673.743429,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
1,2,0.0,10.027499,2673.745801,36246,2673.744167,2673.747430,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
2,3,0.0,13.434188,2673.832800,67384,2673.831165,2673.834429,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
3,4,0.0,15.076840,2673.836801,68816,2673.835166,2673.838430,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
4,5,0.0,12.936264,2673.877601,83419,2673.875967,2673.879230,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,279,0.0,15.029833,2676.444201,1002041,2676.442567,2676.445830,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
279,280,0.0,17.633291,2676.448200,1003472,2676.446565,2676.449829,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
280,281,0.0,15.217426,2676.488000,1017717,2676.486365,2676.489629,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,
281,282,0.0,16.239102,2676.492001,1019149,2676.490366,2676.493630,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,HIP33499_S,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,,,


## Finding Events
Now let's run `turboSETI` on more files from the same observation so we can find interesting events (a hit that occurs across multiple files) via the [ON/OFF method](https://github.com/UCBerkeleySETI/breakthrough/blob/master/GBT/README.md).

Normally it takes a chunk of time to run the algorithm on all of the files, so here is a little script I wrote that keeps `turboSETI` running in the background if executed in a tmux session:

In [5]:
# %load example_script.py
import os
import glob

# glob will create a list of specific files in a directory. In this case, any file ending in .h5.
filelist = glob.glob('*.h5')

# Where turboSETI will store the output files
out_directory = './'

# Iterate over the files in my list
for file in filelist:
    
    # Execute turboSETI in the terminal
    console = 'turboSETI ' + file + ' -M 4 -s 10 -o ' + out_directory
    os.system(console)

Now that we have ran `turboSETI` on six observations, let's use the `find_event_pipeline` to find events that only exist in the on observations. `find_event_pipeline`'s only required parameter is a .lst text file where each line entry is the file location of a `.dat` file. 

Two other important parameters are the filter_threshold and cadence_length. Here is a docstring explaining what filter threshold is:
```
filter_threshold        Specification for how strict the hit filtering will be.
                        There are 3 different levels of filtering, specified by
                        the integers 1, 2, and 3. Filter_threshold = 1 
                        returns hits above an SNR cut, taking into account the
                        check_zero_drift parameter, but without an ON-OFF check.
                        Filter_threshold = 2 returns hits that passed level 1
                        AND that are in at least one ON but no OFFs. 
                        Filter_threshold = 3 returns events that passed level 2
                        AND that are present in *ALL* ONs. Default is 3.
```
cadence_length is the number of files we wish to include in our cadence. By default the value is 6.

In [7]:
from turbo_seti.find_event.find_event_pipeline import find_event_pipeline

# A list of all the .dat files that turboSETI produced
dat_list = ['diced_Parkes_57941_12846_HIP33499_S_fine.dat',
            'diced_Parkes_57941_13194_HIP33499_R_fine.dat',
            'diced_Parkes_57941_13542_HIP33499_S_fine.dat',
            'diced_Parkes_57941_13884_HIP33499_R_fine.dat',
            'diced_Parkes_57941_14233_HIP33499_S_fine.dat',
            'diced_Parkes_57941_14584_HIP33499_R_fine.dat']


# This writes the dat files into a .lst, as required by the find_event_pipeline
with open('dat_files.lst', 'w') as f:
    for item in dat_list:
        f.write("%s\n" % item)

find_event_pipeline('dat_files.lst',
                    filter_threshold = 3,
                    number_in_cadence = 6)


************   BEGINNING FIND_EVENT PIPELINE   **************

Assuming the first observation is an ON
There are 6 total files in the filelist dat_files.lst
therefore, looking for events in 1 on-off set(s)
with a minimum SNR of 10
Present in all A sources with RFI rejection from the off-sources
not including signals with zero drift
saving the output files

***       S       ***

------   o   -------
Loading data...
Loaded 283 hits from diced_Parkes_57941_12846_HIP33499_S_fine.dat
Loaded 330 hits from diced_Parkes_57941_13194_HIP33499_R_fine.dat
Loaded 396 hits from diced_Parkes_57941_13542_HIP33499_S_fine.dat
Loaded 507 hits from diced_Parkes_57941_13884_HIP33499_R_fine.dat
Loaded 559 hits from diced_Parkes_57941_14233_HIP33499_S_fine.dat
Loaded 487 hits from diced_Parkes_57941_14584_HIP33499_R_fine.dat
All data loaded!

Finding events in this cadence...
Found a total of 195 hits above the SNR cut in this cadence!
Found a total of 28 hits in only the on observations in this cadence!
N

[]

No events have been found across the entire cadence, so our ON/OFF method succcessfuly rejected all hits as RFI! Let's change the filter level to 2 and see what we get

In [8]:
event_df = find_event_pipeline('dat_files.lst',
                                filter_threshold = 2,
                                number_in_cadence = 6)
event_df


************   BEGINNING FIND_EVENT PIPELINE   **************

Assuming the first observation is an ON
There are 6 total files in the filelist dat_files.lst
therefore, looking for events in 1 on-off set(s)
with a minimum SNR of 10
Present in at least one A source with RFI rejection from the off-sources
not including signals with zero drift
saving the output files

***       S       ***

------   o   -------
Loading data...
Loaded 283 hits from diced_Parkes_57941_12846_HIP33499_S_fine.dat
Loaded 330 hits from diced_Parkes_57941_13194_HIP33499_R_fine.dat
Loaded 396 hits from diced_Parkes_57941_13542_HIP33499_S_fine.dat
Loaded 507 hits from diced_Parkes_57941_13884_HIP33499_R_fine.dat
Loaded 559 hits from diced_Parkes_57941_14233_HIP33499_S_fine.dat
Loaded 487 hits from diced_Parkes_57941_14584_HIP33499_R_fine.dat
All data loaded!

Finding events in this cadence...
Found a total of 195 hits above the SNR cut in this cadence!
Found a total of 28 hits in only the on observations in this ca

,TopHitNum,DriftRate,SNR,Freq,ChanIndx,FreqStart,FreqEnd,CoarseChanNum,FullNumHitsInRange,FileID,...,MJD,RA,DEC,DELTAT,DELTAF,Hit_ID,status,in_n_ons,RFI_in_range,delta_t
146,147,0.009566,36.482166,2675.224699,565564,2675.223064,2675.226328,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,...,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_1,,0,0.0
683,5,0.009566,10.408674,2673.784800,50204,2673.783165,2673.786428,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
684,6,0.009566,18.400496,2673.787999,51349,2673.786364,2673.789628,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
686,8,0.009566,10.737887,2673.830799,66668,2673.829165,2673.832428,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
715,37,0.009566,17.184115,2674.058399,148129,2674.056764,2674.060028,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
792,114,0.009566,10.957406,2674.473999,296878,2674.472364,2674.475628,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
850,172,0.009566,14.915945,2674.773699,404145,2674.772065,2674.775328,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
852,174,0.009566,26.856294,2674.779399,406185,2674.777765,2674.781028,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
854,176,0.009566,19.598075,2674.784599,408046,2674.782964,2674.786227,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
870,192,0.009566,45.044374,2674.865199,436894,2674.863565,2674.866828,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0


## Plotting events
Now that we have some events, let's plot them

The `plot_event_pipeline` will output the plots as .png's and show them inline. There are two required parameters, a `.csv` outputted by the `find_event_pipeline` and a `.lst` of `.h5` files that will be plotted. I am also setting the user_validation to True because I do not want to accidentally generate 100's of .pngs.

In [15]:
from turbo_seti.find_event.plot_event_pipeline import plot_event_pipeline

# We need to create a .lst of the .h5 files, so lets go ahead and do that:
filelist = glob.glob('./*.h5')

# Write file locations to a .lst
with open('fil_files.lst', 'w') as f:
    for item in filelist:
        f.write("%s\n" % item)

# and finally we plot
plot_event_pipeline('HIP33499_S_f2_snr10.csv',
                    'fil_files.lst',
                    user_validation=True)

plot_event_pipeline: Opened file HIP33499_S_f2_snr10.csv
plot_event_pipeline: source_name=R
plot_event_pipeline: source_name=R
plot_event_pipeline: source_name=R
plot_event_pipeline: source_name=S
plot_event_pipeline: source_name=S
plot_event_pipeline: source_name=S
Plotting some events for:  HIP33499_S
There are 28 total events in the csv file HIP33499_S_f2_snr10.csv
therefore, you are about to make 28 .png files.


Do you wish to proceed with these settings? (y/n):  n


I'm not interested in generating 28 plots, so I'm going to select 10 hits to plot:

In [10]:
event_df.iloc[0:10].to_csv('condensed_fine_f2_snr10.csv') 
# The filename must have the same format as the .csv outputed by find_event_pipeline

event_df.iloc[0:10]

,TopHitNum,DriftRate,SNR,Freq,ChanIndx,FreqStart,FreqEnd,CoarseChanNum,FullNumHitsInRange,FileID,...,MJD,RA,DEC,DELTAT,DELTAF,Hit_ID,status,in_n_ons,RFI_in_range,delta_t
146,147,0.009566,36.482166,2675.224699,565564,2675.223064,2675.226328,0,24321,diced_Parkes_57941_12846_HIP33499_S_fine.h5,...,57941.148680555554,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_1,,0,0.0
683,5,0.009566,10.408674,2673.784800,50204,2673.783165,2673.786428,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
684,6,0.009566,18.400496,2673.787999,51349,2673.786364,2673.789628,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
686,8,0.009566,10.737887,2673.830799,66668,2673.829165,2673.832428,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
715,37,0.009566,17.184115,2674.058399,148129,2674.056764,2674.060028,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
792,114,0.009566,10.957406,2674.473999,296878,2674.472364,2674.475628,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
850,172,0.009566,14.915945,2674.773699,404145,2674.772065,2674.775328,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
852,174,0.009566,26.856294,2674.779399,406185,2674.777765,2674.781028,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
854,176,0.009566,19.598075,2674.784599,408046,2674.782964,2674.786227,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0
870,192,0.009566,45.044374,2674.865199,436894,2674.863565,2674.866828,0,23048,diced_Parkes_57941_14233_HIP33499_S_fine.h5,...,57941.164733796293,6h57m46.6s,-43d42m32s,18.253611,2.793968,,on_table_3,,0,1387.0


In [16]:
%matplotlib inline
plot_event_pipeline('condensed_fine_f2_snr10.csv',
                    'fil_files.lst',
                    user_validation=True)

plot_event_pipeline: Opened file condensed_fine_f2_snr10.csv
plot_event_pipeline: source_name=R
plot_event_pipeline: source_name=R
plot_event_pipeline: source_name=R
plot_event_pipeline: source_name=S
plot_event_pipeline: source_name=S
plot_event_pipeline: source_name=S
Plotting some events for:  HIP33499_S
There are 10 total events in the csv file condensed_fine_f2_snr10.csv
therefore, you are about to make 10 .png files.


Do you wish to proceed with these settings? (y/n):  y



*************************************************
***     The Parameters for This Plot Are:    ****
Target =  HIP33499_S
Bandwidth =  0.0005  MHz
Time Elapsed (inc. Slew) =  -764  s
Middle Frequency =  2675.2247  MHz
Expected Drift =  0.0096  Hz/s
*************************************************
*************************************************

make_waterfall_plots first_file in list: ./diced_Parkes_57941_13884_HIP33499_R_fine.h5
make_waterfall_plots file 0 in list: ./diced_Parkes_57941_13884_HIP33499_R_fine.h5
make_waterfall_plots file 1 in list: ./diced_Parkes_57941_14584_HIP33499_R_fine.h5
make_waterfall_plots file 2 in list: ./diced_Parkes_57941_13194_HIP33499_R_fine.h5
make_waterfall_plots file 3 in list: ./diced_Parkes_57941_14233_HIP33499_S_fine.h5
make_waterfall_plots file 4 in list: ./diced_Parkes_57941_13542_HIP33499_S_fine.h5
make_waterfall_plots file 5 in list: ./diced_Parkes_57941_12846_HIP33499_S_fine.h5

*************************************************
***     The Pa

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

...

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

Great! Now if we check our current directory, we can see the plots generated by the `plot_event_pipeline`.